In [92]:
#New file to vectorise the reviews to perform the data science on them

In [93]:
import pandas as pd
import numpy as np

In [94]:
df = pd.read_csv("../data/review_dataset.csv")
df = pd.DataFrame(df)
df.dropna()

,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review
0,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,have to say this is the best salon ive ever be...,1.652418e+09,2022-05-13 05:07:20,say best salon ive ever 4 weeks lifting acryli...,say best salon ive ever 4 week lifting acryli...
1,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,nian is incredible ive been coming here for a ...,1.661441e+09,2022-08-25 15:22:45,nian incredible ive coming couple months every...,nian incredible ive coming couple month every...
2,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i got a new set of gels from tee who was so so...,1.658412e+09,2022-07-21 14:04:52,got new set gels tee lovely chat helped choose...,got new set gel tee lovely chat helped choose...
3,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i have been coming to this salon since i moved...,1.660734e+09,2022-08-17 10:57:00,coming salon since moved birmingham took find ...,coming salon since moved birmingham took find...
4,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i had my nails done for the first time and i c...,1.659646e+09,2022-08-04 20:41:01,nails done first time say place super cozy gir...,nail done first time say place super cozy gir...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22374,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,just had ford ranger polished so pleased with ...,1.654364e+09,2022-06-04 17:31:51,ford ranger polished pleased results vehicle l...,ford ranger polished pleased result vehicle l...
22375,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,the name says it all fantastic service great p...,1.641225e+09,2022-01-03 15:44:22,name says fantastic service great prices top q...,name say fantastic service great price top qu...
22376,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,a brilliant super hand car wash drove away spa...,1.652984e+09,2022-05-19 18:06:27,brilliant super hand car wash drove away spark...,brilliant super hand car wash drove away spar...
22377,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,always happy when my black and yellow fiesta c...,1.654781e+09,2022-06-09 13:28:00,always happy black yellow fiesta comes gleamin...,always happy black yellow fiesta come gleamin...


In [95]:
'''Unique words separated into a dictionary'''
test_dataset = df["lemmatized review"]
DF = {}
def keywords(dataset):
    for i in range(len(dataset["lemmatized review"])):
        value = dataset["lemmatized review"][i].strip().split(" ")
        for w in value:
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}


keywords(df)
    



In [96]:
for i in DF:
    DF[i] = len(DF[i])

print(len(DF))
DF

21113


{'say': 1048,
 'best': 2292,
 'salon': 1751,
 'ive': 1759,
 'ever': 1210,
 '4': 414,
 'week': 1272,
 'lifting': 28,
 'acrylic': 572,
 'extension': 184,
 'still': 1006,
 'strong': 99,
 'nian': 3,
 'fabulous': 277,
 'perfectionist': 72,
 'id': 479,
 'recommend': 6531,
 'anyone': 1195,
 'want': 848,
 'get': 2577,
 'nail': 4405,
 'done': 3079,
 'professionally': 209,
 'incredible': 237,
 'coming': 595,
 'couple': 392,
 'month': 776,
 'every': 1537,
 'set': 515,
 'amazing': 2365,
 'biab': 40,
 'gel': 722,
 'last': 1095,
 'long': 695,
 'dont': 1090,
 'lift': 96,
 'chip': 83,
 'appointment': 765,
 'always': 4215,
 'cutest': 3,
 'design': 419,
 'would': 5328,
 'highly': 3658,
 'got': 1615,
 'new': 1840,
 'tee': 3,
 'lovely': 2323,
 'chat': 189,
 'helped': 781,
 'choose': 201,
 'colour': 672,
 'also': 1995,
 'saw': 145,
 'well': 1906,
 'really': 3050,
 'happy': 1987,
 'service': 6539,
 'since': 547,
 'moved': 500,
 'birmingham': 55,
 'took': 1094,
 'find': 794,
 'fit': 157,
 'standard': 479,
 '

In [97]:
total_vocab = [x for x in DF]

In [98]:
def review_freq(word):
    c = 0
    try:
        c=DF[word]
    except:
        pass
    return c

In [100]:
from collections import Counter
N = len(df["lemmatized review"])
review = 0
tf_idf = {}
tfidf_dataset = df["lemmatized review"]
for i in range(N):
    tokens = tfidf_dataset[i].strip().split(" ")
    counter = Counter(tokens)
    words_count = len(tokens)

    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = review_freq(token)
        idf = np.log((N+1)/(df+1))
        tf_idf[review,token] = tf*idf
    review +=1






In [104]:
#tf_idf[(0,word)]